## Load Routines for Printing Matrices

In [1]:
using Dates, LinearAlgebra, Statistics, DelimitedFiles

include("printmat.jl")

printlnPs (generic function with 2 methods)

# The Theory: A Covariance Matrix from Betas

The single index model implies that 

$\operatorname*{Cov}(R_{t})=\beta\beta^{\prime}\operatorname*{Var}
(R_{mt})+\Sigma$,

where $\beta$ is an $n\times 1$ vector of betas for $n$ assets. Element $i$ of this vector is typically estimated by OLS of 

$R_{it} = \alpha_i + \beta_i R_{mt} + \varepsilon_{it}$

We can also write element $i,j$ of $\operatorname*{Cov}(R_{t})$ as

$\sigma_{ij} = b_i b_j \operatorname*{Var}(R_{mt}) + \text{Cov}(\varepsilon_{it},\varepsilon_{jt}),$
but where $\text{Cov}(\varepsilon_{it},\varepsilon_{jt}) = 0 \ \text{ if } \ i \neq j$

The next section gives a simple example.

In [2]:
β      = [0.9;1.1]
StdRes = [10;5]/100
VarRm  = 0.15^2
Σ      = diagm(0 => (StdRes.^2))     #diagonal covariance matrix of residuals

println("β, VarRm*100^2 and Σ*100^2: \n")
printmat(β)
printmat(VarRm*100^2)
printmat(Σ*100^2)

CovR = β*β'*VarRm + Σ
println("Covariance matrix*100^2 calculated from betas: ")
printmat(CovR*100^2)

β, VarRm*100^2 and Σ*100^2: 

     0.900
     1.100

   225.000

   100.000     0.000
     0.000    25.000

Covariance matrix*100^2 calculated from betas: 
   282.250   222.750
   222.750   297.250



# Estimating a Single-Index Model on Data

The next section applies the single index model on 5 of the (25) FF portfolios.

## Loading Data

In [3]:
x    = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1,7,13,19,25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                    #no. obs and  no. test assets

(388, 5)

## Regressions

In [4]:
x      = [ones(T) Rme]                   #regressors
β      = fill(NaN,n)
VarRes = fill(NaN,n)
for i = 1:n
    local b_i, residual
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    β[i]      = b_i[2]
    VarRes[i] = var(residual)
end    
println("\nβ coefficients for $n assets, from OLS of Re on constant and Rme")
printmat(β)


β coefficients for 5 assets, from OLS of Re on constant and Rme
     1.341
     1.169
     0.994
     0.943
     0.849



## Using OLS Estimates to Calculate Covariance Matrix

In [5]:
Σ      = diagm(0 => VarRes)       #creates a diagonal covariance matrix from vector
VarRm = var(Rme)
CovR  = β*β'*VarRm + Σ

println("The data are percentage returns (eg. 6.5), so this affects the scaling of variances and covariances")

println("\nCovariance matrix calculated from betas: ")
printmat(CovR)

println("Covariance matrix calculated from data: ")
printmat(cov(Re))

println("Difference between the two: ")
printmat(CovR-cov(Re))

The data are percentage returns (eg. 6.5), so this affects the scaling of variances and covariances

Covariance matrix calculated from betas: 
    73.475    33.232    28.269    26.797    24.146
    33.232    37.371    24.644    23.361    21.049
    28.269    24.644    26.796    19.872    17.906
    26.797    23.361    19.872    25.200    16.973
    24.146    21.049    17.906    16.973    25.335

Covariance matrix calculated from data: 
    73.475    43.024    29.472    25.013    20.188
    43.024    37.371    29.072    25.136    21.407
    29.472    29.072    26.796    23.632    20.981
    25.013    25.136    23.632    25.200    21.533
    20.188    21.407    20.981    21.533    25.335

Difference between the two: 
    -0.000    -9.791    -1.202     1.784     3.958
    -9.791    -0.000    -4.428    -1.775    -0.357
    -1.202    -4.428     0.000    -3.760    -3.075
     1.784    -1.775    -3.760     0.000    -4.560
     3.958    -0.357    -3.075    -4.560     0.000



# A Multi-Index Model

A multi-index model is based on 

$R_{it} =a_{i}+b_{i}^{\prime}I_{t}+\varepsilon_{it}$,

where $b_{i}$ is a $K\times 1$ vector of slope coefficients.

If $\Omega$ is the covariance matrix of the indices $I_t$, then the covariance of
assets $i$ and $j$ is

$\sigma_{ij}=b_{i}^{\prime}\Omega b_{j}  + \text{Cov}(\varepsilon_{it},\varepsilon_{jt}),$
but where $\text{Cov}(\varepsilon_{it},\varepsilon_{jt}) = 0 \ \text{ if } \ i \neq j$


The code below first estimates $b_i$ for $n$ ssets, and then loops over $i=1:n$ and $j=1:n$ to create the entire variance-covariance matrix of the $n$ assets.

In [6]:
x      = [ones(T) Rme RSMB RHML]               #regressors
K      = size(x,2) - 1 
b      = fill(NaN,(K,n))
VarRes = fill(NaN,n)
for i = 1:n
    local b_i, residual
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    b[:,i]    = b_i[2:end]
    VarRes[i] = var(residual)
end    
println("\nOLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] ")
printmat(b)


OLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] 
     1.070     1.080     1.035     1.056     1.041
     1.264     0.768     0.437     0.153    -0.088
    -0.278     0.160     0.487     0.603     0.770



In [7]:
Omega = cov(x[:,2:end])      #covariance matrix of factors
CovR = fill(NaN,(n,n))
for i = 1:n, j = 1:n         #loop over both i and j
    CovR[i,j] = b[:,i]'Omega*b[:,j]
    if i == j
        CovR[i,i] = CovR[i,i] + VarRes[i]
    end
end

println("\nCovariance matrix calculated from betas: ")
printmat(CovR)

println("\nCovariance matrix calculated from data: ")
printmat(cov(Re))

println("Difference between the two: ")
printmat(CovR-cov(Re))

println("Is the multi-index model better than the single-index model?")


Covariance matrix calculated from betas: 
    73.475    41.847    31.050    25.449    18.940
    41.847    37.371    27.384    24.141    20.145
    31.050    27.384    26.796    22.239    20.109
    25.449    24.141    22.239    25.200    20.717
    18.940    20.145    20.109    20.717    25.335


Covariance matrix calculated from data: 
    73.475    43.024    29.472    25.013    20.188
    43.024    37.371    29.072    25.136    21.407
    29.472    29.072    26.796    23.632    20.981
    25.013    25.136    23.632    25.200    21.533
    20.188    21.407    20.981    21.533    25.335

Difference between the two: 
    -0.000    -1.176     1.578     0.436    -1.248
    -1.176    -0.000    -1.688    -0.995    -1.261
     1.578    -1.688     0.000    -1.393    -0.873
     0.436    -0.995    -1.393     0.000    -0.816
    -1.248    -1.261    -0.873    -0.816     0.000

Is the multi-index model better than the single-index model?
